In [1]:
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit
from tqdm import tqdm

In [2]:
hate_orig = pd.read_csv("data/without_aug/train.csv")
hate_aug = pd.read_csv("data/with_aug/train.csv")

In [12]:
hate_orig.shape

(1853, 4)

In [24]:
groups = hate_orig['url']

best_state = 0
min_diff = 1000000
for random_state in tqdm(range(0, 10000)):
    gss = GroupShuffleSplit(n_splits=1, test_size=0.25, random_state=random_state)

    for train_index, test_index in gss.split(hate_orig, groups=groups):
        hate_orig_train = hate_orig.iloc[train_index]
        hate_orig_val = hate_orig.iloc[test_index]

    try:
        count_train = hate_orig_train['category'].value_counts().to_dict()
        count_test = hate_orig_val['category'].value_counts().to_dict()
        diff = {k : count_train[k] - count_test[k] * 9 for k in count_train}
        value_sum = sum(map(abs, diff.values()))
        if value_sum < min_diff:
            best_state = random_state
            min_diff = value_sum
    except:
        continue

print(best_state, min_diff)
gss = GroupShuffleSplit(n_splits=1, test_size=0.25, random_state=best_state)

for train_index, test_index in gss.split(hate_orig, groups=groups):
    hate_orig_train = hate_orig.iloc[train_index]
    hate_orig_val = hate_orig.iloc[test_index]



100%|██████████| 10000/10000 [00:23<00:00, 428.43it/s]

8012 1577


In [25]:
hate_orig_val.shape, hate_orig_train.shape

((343, 4), (1510, 4))

In [28]:
hate_orig_val.to_csv('data/with_aug_ttv/eval.csv', index=False)
hate_orig_train.to_csv('data/with_aug_ttv/train.csv', index=False)

In [31]:
urls_in_val = list(set(hate_orig_val['url'].to_list()))

In [32]:
len(urls_in_val)

90

In [33]:
print(hate_aug.shape)
hate_aug = hate_aug[~hate_aug['url'].isin(urls_in_val)]
print(hate_aug.shape)

(4407, 4)
(3577, 4)


In [34]:
hate_aug.to_csv('data/with_aug_ttv/train.csv', index=False)